In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Preprocess the data
#Drop irrelevant features
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Encode categorical variable
label_encode_gender=LabelEncoder()
data['Gender'] = label_encode_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [11]:
#OneHotEncode Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [17]:
#Combine one hot encoded columns with original data
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis = 1)


KeyError: "['Geography'] not found in axis"

In [18]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [19]:
#Save the encoders and scalar
with open('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [21]:
#Divide the dataset into independant and dependant features
data.head()
X = data.drop('Exited',axis=1)
y = data['Exited']

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale the features
scalar = StandardScaler()
X_train_scale = scalar.fit_transform(X_train)
X_test_scale = scalar.transform(X_test)


In [22]:
X_train_scale

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [23]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

ANN Implementation


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [25]:
X_train = X_train_scale
X_test = X_test_scale

In [27]:
(X_train.shape[1],)

(12,)

In [30]:
#Build ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#HL1 connected with input layer
    Dense(32,activation='relu'),#HL2
    Dense(1,activation='sigmoid')#output
])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [34]:
#compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [48]:
#Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [49]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)#for patience = next 5 epoch there is no improvement then stop

In [50]:
#Train the model

history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test),epochs=100, #X_test and y_test data is used for comparison of accuracy
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3233 - accuracy: 0.8661 - val_loss: 0.3387 - val_accuracy: 0.8545
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3209 - accuracy: 0.8658 - val_loss: 0.3478 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3182 - accuracy: 0.8681 - val_loss: 0.3423 - val_accuracy: 0.8535
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3146 - accuracy: 0.8696 - val_loss: 0.3499 - val_accuracy: 0.8610
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3144 - accuracy: 0.8715 - val_loss: 0.3566 - val_accuracy: 0.8535
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3142 - accuracy: 0.8694 - val_loss: 0.3530 - val_accuracy: 0.8565
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3114 - accuracy: 0.8712 - val_loss: 0.3485 - val_accuracy: 0.8550

In [53]:
# Save the model
model.save('model.h5')

In [54]:
#Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 21260), started 0:07:01 ago. (Use '!kill 21260' to kill it.)

In [ ]:
##Load the Pickle file

